In [1]:
import pandas as pd
import numpy as np

import os

In [2]:
TRANSFORM_FOLDER = '../../DATA/TRANSFORM/entity_details/'
FINAL_FOLDER = '../../DATA/FINAL'

In [3]:
financial_ratios = pd.read_csv('../../DATA/MODEL/financial_ratios_df.csv')

## Joining financial issues indicators

In [5]:
# pomer VI a zavazkov je mensi ako 8, t.j EQ_TL < 0.08
# 2016 : 0.04, 2017 : 0.06, 2018 : 0.08, 
# EQ < 0
# L3 < 1

In [6]:
def process_issues(row):
    EQ_TL = row.EQ_TL
    EQ = row.EQ
    L3 = row.L3
    y = row.year

    if y <= 2016: threat_criterion = 0.04
    elif y == 2017: threat_criterion = 0.06
    else: threat_criterion = 0.08

    if EQ_TL < threat_criterion or EQ < 0 or L3 < 1:
        return 1
    return 0
    

In [8]:
legal_financial_issues_criterion = []
for i, row in financial_ratios.iterrows():
    legal_financial_issues_criterion.append(
        (row.entity_ico,
        row.year,
        process_issues(row))      
    )

financial_issues_criterion_df = pd.DataFrame(legal_financial_issues_criterion, columns=['entity_ico', 'year', 'RUZ_indirect_criterion'])
display(financial_issues_criterion_df)

,entity_ico,year,RUZ_indirect_criterion
0,698.0,2014,1
1,698.0,2015,0
2,698.0,2016,1
3,698.0,2017,1
4,698.0,2018,0
...,...,...,...
708420,54228875.0,2021,0
708421,54268681.0,2021,0
708422,60748940.0,2014,1
708423,60748940.0,2015,1


In [19]:
financial_issues_criterion_info = financial_issues_criterion_df.query('RUZ_indirect_criterion > 0').drop(columns='RUZ_indirect_criterion')
financial_issues_criterion_info['event_type'] = 'RUZ_indirect_criterion'

In [12]:
# files = os.listdir(TRANSFORM_FOLDER)
files = [
 'financial_statements_register_entity_details.csv',
 'insolvency_register.csv',
 'vat_registration_cancellation.csv']

In [13]:
data = dict()
data['entity_details'] = pd.read_csv(TRANSFORM_FOLDER + 'financial_statements_register_entity_details.csv')
data['insolvency_register'] = pd.read_csv(TRANSFORM_FOLDER + 'insolvency_register.csv')
data['vat_registration_cancellation'] = pd.read_csv(TRANSFORM_FOLDER + 'vat_registration_cancellation.csv')

C:\Users\matus.rohal\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3433: DtypeWarning: Columns (13,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
insolvency_info = data['insolvency_register'].copy()
insolvency_info['proceeding_type'] = insolvency_info['proceeding_type'].apply(lambda x: 'ostatne_konania' if x not in ['konkurz', 'restrukturalizacia '] else x)
insolvency_info

insolvency_info.drop_duplicates(['entity_ico', 'proceeding_type', 'proceeding_month'], inplace=True)

insolvency_info = insolvency_info.sort_values('proceeding_month', ascending=False).groupby(['entity_ico', 'proceeding_type', 'proceeding_month']).tail(1)

insolvency_info.rename(columns={'proceeding_type': 'event_type', 'proceeding_month': 'year'}, inplace=True)

insolvency_info = insolvency_info[['entity_ico', 'year', 'event_type']]

insolvency_info['event_type'] = insolvency_info['event_type'].apply(lambda x: 'RU_' + x)
insolvency_info


,entity_ico,year,event_type
0,53051505.0,2023,RU_konkurz
276,45380058.0,2023,RU_konkurz
287,45257337.0,2023,RU_konkurz
286,35964065.0,2023,RU_konkurz
285,44353260.0,2023,RU_konkurz
...,...,...,...
7033,31105424.0,1993,RU_ostatne_konania
7032,31655785.0,1993,RU_ostatne_konania
6995,598321.0,1993,RU_ostatne_konania
7019,31412874.0,1993,RU_ostatne_konania


In [15]:
vat_registration_cancellation_info = data['vat_registration_cancellation'].copy()
# vat_registration_cancellation_info

vat_registration_cancellation_info['entity_ico'] = pd.to_numeric(vat_registration_cancellation_info['entity_ico'], errors = 'coerce')
vat_registration_cancellation_info.query('entity_ico == entity_ico', inplace=True)
vat_registration_cancellation_info = vat_registration_cancellation_info[['entity_ico', 'vat_cancellation_year']].drop_duplicates(['entity_ico'])
vat_registration_cancellation_info['vat_cancellation_year'] = pd.to_numeric(vat_registration_cancellation_info['vat_cancellation_year'])
vat_registration_cancellation_info['event_type'] = 'DPHZ_vat_registration_cancelled'
vat_registration_cancellation_info.rename(columns = {'vat_cancellation_year': 'year'}, inplace=True)
vat_registration_cancellation_info

,entity_ico,year,event_type
0,37712080.0,2020,DPHZ_vat_registration_cancelled
1,37257455.0,2018,DPHZ_vat_registration_cancelled
2,37713256.0,2013,DPHZ_vat_registration_cancelled
3,35125730.0,2020,DPHZ_vat_registration_cancelled
4,37338048.0,2020,DPHZ_vat_registration_cancelled
...,...,...,...
4749,2083477.0,2018,DPHZ_vat_registration_cancelled
4750,50635808.0,2023,DPHZ_vat_registration_cancelled
4751,6199917.0,2020,DPHZ_vat_registration_cancelled
4752,53714521.0,2023,DPHZ_vat_registration_cancelled


In [16]:
ru_entity_details = data['entity_details'].copy()
ru_entity_details.query('financial_statements_list == financial_statements_list', inplace= True)

ru_entity_details = (ru_entity_details
                     .merge(insolvency_info, on = 'entity_ico', how = 'left')
                     .merge(vat_registration_cancellation_info, on = 'entity_ico', how = 'left'))


ru_entity_details['establishment_date'] = pd.to_datetime(ru_entity_details['establishment_date'], errors='coerce')
ru_entity_details['cancellation_date'] = pd.to_datetime(ru_entity_details['cancellation_date'], errors='coerce')
ru_entity_details['cancellation_date_RO'] = pd.to_datetime(ru_entity_details['cancellation_date_RO'], errors='coerce')

ru_entity_details = ru_entity_details.query('establishment_date == establishment_date and id not in [1019370, 1063773, 1068320]')

ru_entity_details = ru_entity_details[['entity_ico', 'establishment_date', 'cancellation_date', 'cancellation_date_RO']]

ru_entity_details['establishment_year'] = ru_entity_details['establishment_date'].apply(lambda x: x.year if x==x else np.nan)
ru_entity_details['cancellation_year'] = ru_entity_details['cancellation_date'].apply(lambda x: x.year if x==x else np.nan)
ru_entity_details['cancellation_year_RO'] = ru_entity_details['cancellation_date_RO'].apply(lambda x: x.year if x==x else np.nan)

ru_entity_details = ru_entity_details[['entity_ico', 'establishment_year', 'cancellation_year', 'cancellation_year_RO']]
ru_entity_details.columns = ['entity_ico', 'RUZ_established', 'RUZ_cancelled', 'RO_cancelled']
ru_entity_events = ru_entity_details.melt(id_vars='entity_ico')
ru_entity_events.columns = ['entity_ico', 'event_type', 'year']
ru_entity_events.query('year == year', inplace = True)
ru_entity_events['year'] = ru_entity_events['year'].apply(lambda x: x if isinstance(x, float) else x.year)
ru_entity_events

,entity_ico,event_type,year
0,603783.0,RUZ_established,1991.0
1,590797.0,RUZ_established,1990.0
2,2313.0,RUZ_established,1992.0
3,677957.0,RUZ_established,1992.0
4,615439.0,RUZ_established,1991.0
...,...,...,...
706969,52425576.0,RO_cancelled,2021.0
707009,52618382.0,RO_cancelled,2021.0
707059,52835383.0,RO_cancelled,2022.0
707201,53367600.0,RO_cancelled,2021.0


In [31]:
valid_ico_list = list(financial_ratios['entity_ico'].unique())

In [32]:
entity_issues_events_long = (pd.concat(
    [
         insolvency_info,
         ru_entity_events,
         vat_registration_cancellation_info,
         financial_issues_criterion_info
    ], axis = 0))

entity_issues_events_long['event_type'].value_counts()

RUZ_established                    349332
RUZ_indirect_criterion             262131
RUZ_cancelled                       55828
RU_konkurz                           5763
DPHZ_vat_registration_cancelled      4753
RO_cancelled                         2200
RU_ostatne_konania                   1269
Name: event_type, dtype: int64

In [39]:
entity_issues_events_long.query('event_type not in ["RUZ_established", "RO_cancelled"] and entity_ico in @valid_ico_list and year >= 2014', inplace=True)

entity_issues_events_long['event_type'].value_counts()

RUZ_indirect_criterion             262131
RUZ_cancelled                        9822
DPHZ_vat_registration_cancelled      2978
RU_konkurz                           2123
RU_ostatne_konania                    359
Name: event_type, dtype: int64

In [40]:
entity_issues_events_long['indicator'] = 1
entity_issues_events_long.to_csv('../../DATA/TRANSFORM/issues_events.csv', index = False)

In [48]:

entity_issues_indicators_1y_off = entity_issues_events_long.copy()
entity_issues_indicators_2y_off = entity_issues_events_long.copy()

entity_issues_indicators_1y_off['year'] = entity_issues_indicators_1y_off['year'] - 1
entity_issues_indicators_1y_off['event_type'] = entity_issues_indicators_1y_off['event_type'].apply(lambda x: x+'_1y_off')

entity_issues_indicators_2y_off['year'] = entity_issues_indicators_2y_off['year'] - 2
entity_issues_indicators_2y_off['event_type'] = entity_issues_indicators_2y_off['event_type'].apply(lambda x: x+'_2y_off')

entity_issues_indicators = (pd.concat(
    [
        entity_issues_events_long.copy(),
        entity_issues_indicators_1y_off,
        entity_issues_indicators_2y_off
    ], axis=0
))

entity_issues_events = pd.pivot_table(entity_issues_indicators, index = ['entity_ico', 'year'], columns = 'event_type', values='indicator', fill_value=0)
entity_issues_events.reset_index(inplace=True)
entity_issues_events

event_type,entity_ico,year,DPHZ_vat_registration_cancelled,DPHZ_vat_registration_cancelled_1y_off,DPHZ_vat_registration_cancelled_2y_off,RUZ_cancelled,RUZ_cancelled_1y_off,RUZ_cancelled_2y_off,RUZ_indirect_criterion,RUZ_indirect_criterion_1y_off,RUZ_indirect_criterion_2y_off,RU_konkurz,RU_konkurz_1y_off,RU_konkurz_2y_off,RU_ostatne_konania,RU_ostatne_konania_1y_off,RU_ostatne_konania_2y_off
0,698.0,2012.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,698.0,2013.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,698.0,2014.0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
3,698.0,2015.0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
4,698.0,2016.0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496460,54192293.0,2021.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
496461,60748940.0,2012.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
496462,60748940.0,2013.0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
496463,60748940.0,2014.0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0


In [50]:
ISSUES_TYPES = ['DPHZ_vat_registration_cancelled', 'RUZ_cancelled','RUZ_indirect_criterion', 'RU_konkurz', 'RU_ostatne_konania']

In [53]:
def process_entity_issues(row, issue_type):
        return max(row[issue_type+'_1y_off'], row[issue_type+'_2y_off'])

for issue_type in ISSUES_TYPES:
    entity_issues_events[issue_type+'_2y_off'] = entity_issues_events.apply(lambda x: process_entity_issues(x, issue_type), axis = 1)

entity_issues_events

event_type,entity_ico,year,DPHZ_vat_registration_cancelled,DPHZ_vat_registration_cancelled_1y_off,DPHZ_vat_registration_cancelled_2y_off,RUZ_cancelled,RUZ_cancelled_1y_off,RUZ_cancelled_2y_off,RUZ_indirect_criterion,RUZ_indirect_criterion_1y_off,RUZ_indirect_criterion_2y_off,RU_konkurz,RU_konkurz_1y_off,RU_konkurz_2y_off,RU_ostatne_konania,RU_ostatne_konania_1y_off,RU_ostatne_konania_2y_off
0,698.0,2012.0,0,0,0.0,0,0,0.0,0,0,1.0,0,0,0.0,0,0,0.0
1,698.0,2013.0,0,0,0.0,0,0,0.0,0,1,1.0,0,0,0.0,0,0,0.0
2,698.0,2014.0,0,0,0.0,0,0,0.0,1,0,1.0,0,0,0.0,0,0,0.0
3,698.0,2015.0,0,0,0.0,0,0,0.0,0,1,1.0,0,0,0.0,0,0,0.0
4,698.0,2016.0,0,0,0.0,0,0,0.0,1,1,1.0,0,0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496460,54192293.0,2021.0,0,0,0.0,0,0,0.0,1,0,0.0,0,0,0.0,0,0,0.0
496461,60748940.0,2012.0,0,0,0.0,0,0,0.0,0,0,1.0,0,0,0.0,0,0,0.0
496462,60748940.0,2013.0,0,0,0.0,0,0,0.0,0,1,1.0,0,0,0.0,0,0,0.0
496463,60748940.0,2014.0,0,0,0.0,0,0,0.0,1,1,1.0,0,0,0.0,0,0,0.0


In [56]:
display(financial_ratios.shape[0])

financial_ratios_copy = financial_ratios.merge(entity_issues_events, on = ['entity_ico', 'year'], how = 'left')
display(financial_ratios.shape[0])


708425

708425

In [57]:
financial_ratios_copy.to_csv('../../DATA/MODEL/training_data_temp.csv', index = False)

In [58]:
entity_issues_indicators = entity_issues_indicators.query('entity_ico > 0')[['entity_ico', 'year'] + sorted(list(entity_issues_indicators.columns)[2:])].sort_values(['entity_ico', 'year'])

In [59]:
entity_issues_indicators.to_csv('../../DATA/TRANSFORM/issues_indicators.csv', index = False)